In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.remote.webelement import WebElement
import pandas as pd
import yaml
from bs4 import BeautifulSoup as bs
import requests
from uuid import UUID

# from src.utils.data_log_config import logger


#### Get news links

In [ ]:
page_no = 0
url = f"https://www.fxstreet.com/news/latest/asset?q=&hPP=17&idx=FxsIndexPro&p={page_no}&dFR%5BCategory%5D%5B0%5D=News&dFR%5BTags%5D%5B0%5D=EURUSD"

In [3]:
wd = webdriver.Chrome()
page = wd.get(url)

In [4]:
news_boxes = wd.find_elements(
    By.CLASS_NAME,
    "ais-hits--item"
)

In [5]:
len(news_boxes)

17

In [6]:
news_box = news_boxes[0]

def get_news_data(news_box: WebElement) -> dict:
    headline = news_box.find_element(
        By.CLASS_NAME,
        "fxs_headline_tiny"
    )
    headline_text = headline.text
    link = headline.find_element(
        By.TAG_NAME,
        "a"
    ).get_attribute("href")
    date_time = news_box.find_element(
        By.TAG_NAME,
        "time"
    ).get_attribute("datetime")

    return {
        "date_time": date_time,
        "headline": headline_text,
        "link": link,
    }


In [7]:
news_data = get_news_data(news_box)

In [8]:
news_data["date_time"][:10]

'2024-11-22'

#### Get article text

In [9]:
def get_article_text(article_link: str) -> str:
    news_article = requests.get(article_link)
    article_page = bs(news_article.content, "html.parser")
    return article_page.find('div', class_='fxs_article_body').text

In [10]:
article_text = get_article_text(news_data["link"])

#### Page parsing logic

In [13]:
page_no = 18
url = f"https://www.fxstreet.com/news/latest/asset?q=&hPP=17&idx=FxsIndexPro&p={page_no}&dFR%5BCategory%5D%5B0%5D=News&dFR%5BTags%5D%5B0%5D=EURUSD"

In [ ]:
def scrape_news_data(url: str) -> list:
    wd = webdriver.Chrome()
    news_data_list = []
    page_no = 0
    url = f"https://www.fxstreet.com/news/latest/asset?q=&hPP=17&idx=FxsIndexPro&p={page_no}&dFR%5BCategory%5D%5B0%5D=News&dFR%5BTags%5D%5B0%5D=EURUSD"

    if len(news_data) == 0:
        _ = wd.get(url)
        news_boxes = wd.find_elements(
            By.CLASS_NAME,
            "ais-hits--item"
        )
        for news_box in news_boxes:
            news_data_list.append(get_news_data(news_box))
            page_no += 1

    while True:
    
        if news_data_list[-1]["date_time"][:10] != "2013-12-31":
            break
        

In [18]:
news_data_list = [news_data]
news_data_list[-1]["date_time"][:10] == "2024-11-22"

True